In [ ]:
from google.colab import drive # import drive from google colab
from os.path import join  

In [ ]:
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

In [ ]:
%cd '/content/drive'
directory_name = !find . -type d -name "ucl-nlp-finalproject"
directory_name = directory_name[0]
print(directory_name)

In [ ]:
%cd $directory_name

In [ ]:
import sys
sys.path.append('..')
import numpy as np
import pickle
import waku
from waku.tasks.sentiment_analysis.sentiment_analysis_downstream import SST, Extrinsic_Sentiment_Analysis
from waku.tasks.word_intrusion.word_intrusion import top_ten, dist_ratio

In [ ]:
# file paths must be .pickle and .npy
dict_filepath = 'some_filepath.pickle'
weights_filepath = 'some_other_filepath.npy'

# folder path for saving experiments
EXPERIMENT_FOLDER_PATH = ""
NAME = "vanilla_300"

# assert EXPERIMENT_FOLDER_NAME in ['hoyer_1000', 'hoyer_300', 'spine_1000_1000', 'spine_300_1000', 'spine_300_300', 'vanilla_1000', 'vanilla_300']
# assert len(EXPERIMENT_FOLDER_NAME) != 0

In [ ]:
# read pickle file into dictionary
def load_pickle(filepath):
    # file path must end with .pickle
    pickle_in = open(filepath,"rb")
    emb_dict = pickle.load(pickle_in)
    return emb_dict

# load using helper function
embedding_dict = load_pickle(dict_filepath)
embedding_weights = np.load(weights_filepath)['a']

Run sentiment analysis experiment:

In [ ]:
# Instantiate a *SST* to load Stanford Sentiment Treebank train/test/val data
SST_instance = SST()

# Load an instance of the *Extrinsic_Sentiment_Analysis* class with a given dictionary and 
# weights at which point we reduce the vocabulary into words present in SST
Experiment = Extrinsic_Sentiment_Analysis(SST_instance, embedding_dict, embedding_weights)

# Train a specified LSTM model for a given number of epochs using the *train* function
Experiment.train(epochs=50, learning_rate=0.001, batch_size=512, hidden_size=300, rnn_layers=2, mlp_layer_widths=100)

# Calculate accuracy on the test set and save
Experiment.test(print_accuracies=True, save_test_acc=True, file_path=EXPERIMENT_FOLDER_PATH, file_name=NAME)

# reset data class
SST_instance.reset()

Word intrusion experiment:

In [ ]:
top_ten = top_ten_set(embedding_weights)
dist_ratio(embedding_weights, top_ten, 5, 10, print_result=True, save_acc=True, file_path=EXPERIMENT_FOLDER_NAME, file_name=NAME)